In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
#import seaborn as sns
import pymongo
from pprint import pprint

In [2]:
client = pymongo.MongoClient()
db = client.metrics

In [4]:
from bson.son import SON # needed to ensure dictionary is ordered (python default is not)
import hashlib

def hash_feats(fts):
    vals = fts.values
    joined = "|".join(map(lambda s: str(s),vals)).encode('utf-8') 
    return hashlib.sha224(joined).hexdigest()

def get_df_sorted_by_f1score(collection, params=None, filter_cols=True):
    if not params:
        params = []
    if type(params) == str:
        params = params.split(",")
    
    project = {
            "weighted_f1_score":"$WEIGHTED_MEAN_CONCEPT_CODES.f1_score",
            "macro_f1_score":   "$MACRO_F1",
            "micro_f1_score":  "$MICRO_F1.f1_score",
            "micro_recall":    "$MICRO_F1.recall",
            "micro_precision": "$MICRO_F1.precision",
    
    # PARAMETERS            
            "window_size":    "$parameters.window_size",
            "feats":          "$parameters.extractors",
            "count": {        "$size" : "$parameters.extractors" },
            "asof" :          "$asof",
            "_id":1
    }
    
    # No count for HMM
    if "_hmm" in collection.lower():
        del project["count"]
    
    for param in params:
        project[param] = "$parameters." + param

    feats_pipeline = [{
        "$project": project
    },
    {
        "$match":{
            "micro_f1_score": { "$exists" : True }        
        }
    },
    {
        "$sort":{
            "micro_f1_score": -1
        }
    },
    ]
    
    rows = [row for row in db[collection].aggregate(feats_pipeline)]
    df = pd.DataFrame(rows).sort_values("micro_f1_score", ascending=False)
    if params:
        df["hs_params"] = df[params].apply(hash_feats, axis=1)
        
    if filter_cols:
        cols = ["micro_f1_score", "micro_recall" ,"micro_precision", "macro_f1_score" ] + params
        return df[cols]
    return df

In [6]:
def round_data(df, places=3):
    df_copy = df.copy()
    fmt_str = "{0:." + str(places) + "f}"
    cols = set([v for v in df_copy.columns.values if "micro_" in v])
    for c in cols:
        df_copy[c] = df[c].apply(lambda d: fmt_str.format(d))  
    return df_copy

### What Experiments Do We Currently Have Data For?

#### CB

In [85]:
cb_collections = \
[str(col)
 for col 
 in sorted(db.collection_names())
 if 
     "CB_" in col and 
     "CR_" in col and 
     "VD" in col and
     "TEST" not in col and 
 1==1]
cb_collections

['CR_CB_SHIFT_REDUCE_PARSER_TEMPLATED_VD',
 'CR_CB_SHIFT_REDUCE_PARSER_VD',
 'CR_CB_STACKED_VD',
 'CR_CB_TAGGING_VD_MOST_COMMON_TAG_RNN',
 'CR_CB_TAGGING_VD_RNN']

[str(col)
 for col 
 in sorted(db.collection_names())
 if 
     "CB_" in col and 
     "CR_" in col and 
     "VD" in col and
     "TEST" in col and 
 1==1]

#### SC

In [86]:
sc_collections = \
[str(col)
 for col 
 in sorted(db.collection_names())
 if 
     "SC_" in col and 
     "CR_" in col and 
     "VD" in col and
     "TEST" not in col and 
 1==1]
sc_collections

['CR_SC_STACKED_VD', 'CR_SC_TAGGING_VD_MOST_COMMON_TAG_RNN']

#### - The 'Stacked' and 'Most Common Tag' Are the Initial 2 Model Types (Tagging without Most Common was a Quick Experiment)

In [52]:
def compute_count_codes(row):

    keys = []
    unique = set()
    for k in row.keys():        
        if "->" in k: #k[0].isdigit():
            keys.append(k)
            unique.add(k)
    assert len(keys) == len(unique), "Duplicate keys found"
    return unique

In [80]:
sc_collections

['CR_SC_STACKED_VD', 'CR_SC_TAGGING_VD_MOST_COMMON_TAG_RNN']

In [84]:
COLLECTION = [c for c in cb_collections] #if  1==1 or "STACKED" not in c]

col2codes = {}
for col in COLLECTION:
    cursor = db[col].find({})
    print(col, cursor.count())
    first_row_codes = None
    for row in cursor:
        codes = compute_count_codes(row)
        if not first_row_codes:
            first_row_codes = set(codes)
            col2codes[col] = codes
        else:            
            assert first_row_codes == codes, "Different codes computed across different runs"

print("")
for col,codes in col2codes.items():
    print(col, len(codes))
            
print("")
for col, codes in col2codes.items():
    for col2, codes2 in col2codes.items():
        if col >= col2:
            continue
        
        if codes != codes2:
            diff_left  = codes - codes2
            diff_right = codes2 - codes

            print("Different codes computed for collections: 1: {col1} [{count_1}] and 2: {col2} [{count_2}]"\
                  .format(col1=col, count_1=len(codes), col2=col2, count_2=len(codes2)))
            
            print("Left Count: {lc} Right Count: {rc}".format(lc=len(diff_left), rc=len(diff_right)))
            print("\tLEFT:  {left}\n\tRIGHT: {right}".format(left="\t".join(diff_left), right="\t".join(diff_right)))
            print("")

('CR_CB_SHIFT_REDUCE_PARSER_TEMPLATED_VD', 1)
('CR_CB_SHIFT_REDUCE_PARSER_VD', 6)
('CR_CB_STACKED_VD', 49)
('CR_CB_TAGGING_VD_MOST_COMMON_TAG_RNN', 6)
('CR_CB_TAGGING_VD_RNN', 6)

('CR_CB_SHIFT_REDUCE_PARSER_TEMPLATED_VD', 82)
('CR_CB_SHIFT_REDUCE_PARSER_VD', 82)
('CR_CB_STACKED_VD', 88)
('CR_CB_TAGGING_VD_MOST_COMMON_TAG_RNN', 82)
('CR_CB_TAGGING_VD_RNN', 82)

Different codes computed for collections: 1: CR_CB_SHIFT_REDUCE_PARSER_TEMPLATED_VD [82] and 2: CR_CB_STACKED_VD [88]
Left Count: 0 Right Count: 6
	LEFT:  
	RIGHT: Causer:Change-factor->Result:50	Causer:6->Result:5	Causer:7->Result:13	Causer:4->Result:Change-factor	Causer:5->Result:11	Causer:7->Result:1

Different codes computed for collections: 1: CR_CB_SHIFT_REDUCE_PARSER_VD [82] and 2: CR_CB_STACKED_VD [88]
Left Count: 0 Right Count: 6
	LEFT:  
	RIGHT: Causer:Change-factor->Result:50	Causer:6->Result:5	Causer:7->Result:13	Causer:4->Result:Change-factor	Causer:5->Result:11	Causer:7->Result:1

Different codes computed for colle